### Dummy Data

In [65]:
import json

In [103]:
with open('data_train (2).json', 'r') as file:
    candidates = json.load(file)

In [104]:
jobreq = "deskripsi kerja posisi lowong partisipasi prose bangun aplikasi sedang bangun usaha nama usaha harap mampu kerja tim responsibility buat modifikasi program tanggung jawab kelola program"

In [5]:
!pip install -q googletrans==4.0.0-rc1
!pip install -q googletrans==4.0.0-rc1 langdetect

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [105]:
from googletrans import Translator
from langdetect import detect

In [106]:
def translate_text(text):
    translator = Translator()
    try:
        # Detect the language
        lang = detect(text)
        # Translate only if the text is in Indonesian
        if lang == 'id':
            translated = translator.translate(text, src='id', dest='en')
            return translated.text
        else:
            # Return the original text if it's not in Indonesian
            return text
    except Exception as e:
        # If detection or translation fails, return the original text
        return text

In [107]:
from datetime import datetime

In [108]:
def calculate_duration(start_date, end_date):
    if start_date.lower() in ['none', 'unknown']:
        return "Unknown duration"

    # Try parsing the start_date in different formats
    try:
        start_date_obj = datetime.strptime(start_date, "%b %Y")
    except ValueError:
        try:
            start_date_obj = datetime.strptime(start_date, "%Y")
        except ValueError:
            return "Unknown duration"

    if end_date.lower() == 'present':
        end_date_obj = datetime.now()
    elif end_date.lower() in ['none', 'unknown']:
        return "Unknown duration"
    else:
        # Try parsing the end_date in different formats
        try:
            end_date_obj = datetime.strptime(end_date, "%b %Y")
        except ValueError:
            try:
                end_date_obj = datetime.strptime(end_date, "%Y")
            except ValueError:
                return "Unknown duration"

    # Calculate the difference between end_date and start_date
    duration = end_date_obj - start_date_obj

    # Extract years and months from the duration
    years = duration.days // 365
    months = (duration.days % 365) // 30

    # Format the duration
    if years >= 1:
        if years == 1:
            return "+1 year"
        elif years == 2:
            return "+2 years"
        else:
            return f"+{years} years"
    else:
        return f"{months} months"

In [109]:
# Function to translate candidates' data
def translate_candidate_data(candidate):
    # Translate work experience descriptions
    for experience in candidate.get("work_experience", []):
        # Translate job title
        job_title = experience.get("job_title", "")
        if job_title:
            translated_title = translate_text("".join(job_title))
            experience["job_title"] = translated_title

        # Translate job description
        if experience.get("job_desc"):
            experience["job_desc"] = [translate_text(desc) for desc in experience["job_desc"]]

        # Calculate duration
        duration = calculate_duration(experience.get("start_date", ""), experience.get("end_date", ""))
        experience["duration"] = duration

    # Translate education title and description if they exist
    for education in candidate.get("education", []):
        if isinstance(education, dict):
            # Translate education title
            education_title = education.get("title", "")
            if education_title:
                translated_education_title = translate_text("".join(education_title))
                education["title"] = translated_education_title

            # Translate education description
            if education.get("description"):
                education["description"] = translate_text(education["description"])

    # Translate languages
    candidate["languages"] = [translate_text(language) for language in candidate.get("languages", [])]

    # Translate skills
    candidate["skills"] = [translate_text(skill) for skill in candidate.get("skills", [])]


In [110]:
for candidate in candidates:
    translate_candidate_data(candidate)

In [ ]:
# Output the modified JSON for candidates
print(json.dumps(candidates, indent=5))

In [75]:
def translate_job_requirements(jobreq):
    translated_jobreq = {}

    # Translate the entire job requirements string
    translated_jobreq["job_requirements"] = translate_text(jobreq)

    return translated_jobreq

In [76]:
translate_jobreq = translate_job_requirements(jobreq)

In [ ]:
# Output the modified JSON for job requirements
print(json.dumps(translate_jobreq, indent=4))

In [112]:
import tensorflow as tf
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
!pip -q install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.6 MB/s eta 0:00:00


In [19]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [113]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Initialize stopwords and lemmatizer
english_stop_words = set(stopwords.words('english'))
stopword_factory = StopWordRemoverFactory()
indonesian_stop_words = set(stopword_factory.get_stop_words())
all_stop_words = english_stop_words.union(indonesian_stop_words)
lemmatizer = WordNetLemmatizer()

In [114]:
def text_preprocessing(text):
    if text is None:
        return ''

    if isinstance(text, dict):
        text = preprocess_job_requirements(text)

    text_no_punct = ''.join([char for char in text if char not in string.punctuation])
    text_lower = text_no_punct.lower()
    text_single_line = ' '.join(text_lower.split())
    tokens = word_tokenize(text_single_line)
    tokens_cleaned = [lemmatizer.lemmatize(word) for word in tokens if word not in all_stop_words]
    clean_text = ' '.join(tokens_cleaned)

    return clean_text

In [115]:
def preprocess_job_requirements(jobreq):
    # Preprocess the entire job requirements string
    preprocessed_jobreq = text_preprocessing(jobreq)

    return {
        'job_requirements': preprocessed_jobreq
    }

In [116]:
clean_jobreq = preprocess_job_requirements(translate_jobreq["job_requirements"])

In [ ]:
clean_jobreq

In [118]:
def preprocess_candidate(candidate):
    work_experience = []
    for experience in candidate.get('work_experience', []):
        preprocessed_experience = {
            'job_title': text_preprocessing(experience.get('job_title', '')),
            'company': text_preprocessing(experience.get('company', '')),
            'location': text_preprocessing(experience.get('location', '')),
            'start_date': text_preprocessing(experience.get('start_date', '')),
            'end_date': text_preprocessing(experience.get('end_date', '')),
            'job_desc': [text_preprocessing(desc) for desc in experience.get('job_desc', [])],
            'duration': text_preprocessing(experience.get('duration', ''))
        }
        work_experience.append(preprocessed_experience)

    education = []
    for edu in candidate.get('education', []):
        if isinstance(edu, dict):
            preprocessed_edu = {
                'title': text_preprocessing(edu.get('title', '')),
                'institute': text_preprocessing(edu.get('institute', '')),
                'location': text_preprocessing(edu.get('location', '')),
                'start_date': text_preprocessing(edu.get('start_date', '')),
                'end_date': text_preprocessing(edu.get('end_date', '')),
                'description': text_preprocessing(edu.get('description', ''))
            }
            education.append(preprocessed_edu)

    languages = [text_preprocessing(language) if language != 'Indonesia' else language for language in candidate.get('languages', [])]
    skills = [text_preprocessing(skill) for skill in candidate.get('skills', [])]

    certification = []
    for cert in candidate.get('certification', []):
        if isinstance(cert, dict):
            preprocessed_cert = {
                'title': text_preprocessing(cert.get('title', '')),
                'issuer': text_preprocessing(cert.get('issuer', '')),
                'start_date': text_preprocessing(cert.get('start_date', '')),
                'expiration_date': text_preprocessing(cert.get('expiration_date', ''))
            }
            certification.append(preprocessed_cert)

    return {
        'basic_info': candidate.get('basic_info', {}),
        'work_experience': work_experience,
        'education': education,
        'languages': languages,
        'skills': skills,
        'certification': certification
    }

In [119]:
def preprocess_candidates(candidates):
    preprocessed_candidates = []
    for candidate in candidates:
        preprocessed_candidate = preprocess_candidate(candidate)
        preprocessed_candidates.append(preprocessed_candidate)
    return preprocessed_candidates


In [120]:
clean_candidates = preprocess_candidates(candidates)

In [ ]:
print(json.dumps(clean_candidates, indent=4))

In [122]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = text.replace('unknown', '')
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words)
    return text

def preprocess_candidate_data(candidate):
    combined_text = []
    
    # Combine basic info
    basic_info = candidate.get('basic_info', {})
    combined_text.append(basic_info.get('location', ''))

    # Combine work experience
    for work in candidate.get('work_experience', []):
        if isinstance(work, dict):
            combined_text.append(work.get('job_title', ''))
            combined_text.append(work.get('company', ''))
            combined_text.append(work.get('location', ''))
            combined_text.append(' '.join(work.get('job_desc', [])))

    # Combine education
    for education in candidate.get('education', []):
        if isinstance(education, dict):
            combined_text.append(education.get('title', ''))
            combined_text.append(education.get('institute', ''))
            combined_text.append(education.get('location', ''))
            combined_text.append(education.get('description', ''))

    # Combine languages, skills, and certifications
    combined_text.extend(candidate.get('languages', []))
    combined_text.extend(candidate.get('skills', []))
    for certification in candidate.get('certification', []):
        if isinstance(certification, dict):
            combined_text.append(certification.get('title', ''))
            combined_text.append(certification.get('issuer', ''))

    # Ensure all elements in combined_text are strings
    combined_text = ' '.join(map(str, combined_text))
    return preprocess_text(combined_text)

# Preprocess job requirements
preprocessed_jobreq = preprocess_text(jobreq)

In [ ]:
# Translate and preprocess job requirement
translated_jobreq = translate_text(jobreq)
preprocessed_jobreq = preprocess_text(translated_jobreq)
print(preprocessed_jobreq)

In [ ]:
# Translate and preprocess all candidates
for candidate in candidates:
    translate_candidate_data(candidate)
preprocessed_candidates = [preprocess_candidate_data(candidate) for candidate in candidates]

# Print the preprocessed candidates
for i, preprocessed_candidate in enumerate(preprocessed_candidates):
    print(f"Candidate {i+1}:")
    print(preprocessed_candidate)
    print()

In [125]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer

In [126]:
# Create a dummy jobreq_data DataFrame
jobreq_data = pd.DataFrame({
    'description': [preprocessed_jobreq]
})

# Assuming preprocessed_candidates is a list of preprocessed candidate strings
cv_data = pd.DataFrame({
    'Text': preprocessed_candidates
})

In [ ]:
jobreq_data

In [ ]:
cv_data

In [129]:
# Load the saved model
model = load_model('collaborative_filtering_model_newest.h5')

In [ ]:
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the CV data
vectorizer.fit(cv_data['Text'])

# Transform the job requirement description
jobreq_tfidf = vectorizer.transform(jobreq_data['description'])

# Define a function to recommend CVs for a job requirement using the trained collaborative filtering model
def recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data, top_n=5):
    # Transform the job description to its TF-IDF representation
    jobreq_tfidf = vectorizer.transform([jobreq_data.loc[jobreq_id, 'description']])
    
    # Get the indices of CVs
    cv_indices = np.arange(len(cv_data))
    
    # Repeat the job requirement index for all CVs
    job_indices = np.repeat(jobreq_id, len(cv_data))
    
    # Predict the relevance scores using the model
    predicted_points = model.predict([cv_indices, job_indices])
    
    # Get the top recommended CVs
    top_cvs_indices = predicted_points.squeeze().argsort()[::-1][:top_n]
    
    # Get the CV indices (acting as CV IDs), texts, and similarity scores
    top_cvs_with_scores = []
    for cv_idx in top_cvs_indices:
        cv_id = cv_idx  # Use the row index as the CV ID
        cv_text = cv_data.loc[cv_idx, 'Text']
        similarity_score = predicted_points[cv_idx].item()  # Convert to a scalar value
        top_cvs_with_scores.append((cv_id, cv_text, similarity_score))
    
    return top_cvs_with_scores

# Example: get top 5 CVs for a specific job requirement using the collaborative filtering model
jobreq_id = 0  # Replace with the appropriate job requirement index

top_cvs_with_scores = recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data)

print("Top CVs for Job Requirement ID {}: ".format(jobreq_id))
for cv_id, cv_text, similarity_score in top_cvs_with_scores:
    print(f"CV ID: {cv_id}\nCV: {cv_text}\nSimilarity Score: {similarity_score}\n")

# Store the results in a dictionary
result_dict = {
    "jobreq_id": jobreq_id,
    "recommendations": [
        {
            "cv_id": cv_id,
            "cv_text": cv_text,
            "similarity_score": similarity_score
        }
        for cv_id, cv_text, similarity_score in top_cvs_with_scores
    ]
}

# Write the dictionary to a JSON file
output_file = 'recommended_cvs.json'
with open(output_file, 'w') as f:
    json.dump(result_dict, f, indent=4)

print(f"Results saved to {output_file}")